<a href="https://colab.research.google.com/github/Farmers-For-Forests/treedetection/blob/main/End_to_End_to_share.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import logging
log = logging.getLogger("pytorch_lightning")
log.propagate = False
log.setLevel(logging.ERROR)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# @ Vijen
# this code wont be needed for deployment but to check it before sending it to you,
# I have it included here
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title
%%capture
#install the package, on colab make sure to upgrade existing packages. This is not needed in a clean env.
!pip install --upgrade deepforest
!pip install git+https://github.com/weecology/DeepForest.git
!pip install -Uqq ipdb
!pip install utm

# importing Libraries (Some of them are probably not used and I will clean it later)

In [ ]:
import io
import glob
import os
import sys
import imghdr
from deepforest import utilities
from deepforest import get_data
from deepforest import main
from deepforest import preprocess
from deepforest import visualize
from deepforest import evaluate
import rasterio
import rasterio.warp
from rasterio import CRS
import cv2
import matplotlib.pyplot as plt
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1347753768
import pandas as pd
import numpy as np
import tempfile
import math
import pyproj
from pyproj import Proj
from keras.models import load_model
from pickle import load
from os import path
plt.rcParams['figure.figsize'] = [10,15]

# Function Definitions

In [ ]:
#### Used in Prediction ####

def PredictTrees(model, image, patch_size):
    '''
    Arguments:  1. model: deepforest tree detection model
    Description:  1. First, the number of parallel processes are set. This is dependent on available compute. Chnage this number if not applicable
                  2. predict a datafram with deepforest
                  3. predict an image with annotation rectangles from deepforest prediction
                  ( in the predict_tile function used from deepforest model, patch_size, patch_overlap, thickness are hardcoded in the function below. Change them or function could be changed to pass these)
    Returns:  1. predicted_df: pandas dataframe
              2. predicted image: an image object which is an RGB array (check if it is RGB or BGR)
    '''

    model.config["workers"] = 8

    predicted_df = model.predict_tile(image, return_plot = False, patch_size=patch_size, patch_overlap=0.25, thickness=10)
    predicted_image = model.predict_tile(image, return_plot = True, patch_size=patch_size, patch_overlap=0.25, thickness=10)


    return predicted_df, predicted_image

#### Used in getting height from DEM ####

def GetTreeHeight(row, dem_file, image_file, dem_data):
  xmin = row[0]
  ymin = row[1]
  xmax = row[2]
  ymax = row[3]
    # print(xmin, ymin, xmax, ymax)

  try:
    xdem_min = xmin * (rasterio.open(dem_file).width/rasterio.open(image_file).width)
    ydem_min = ymin * (rasterio.open(dem_file).height/rasterio.open(image_file).height)
    xdem_max = xmax * (rasterio.open(dem_file).width/rasterio.open(image_file).width)
    ydem_max = ymax * (rasterio.open(dem_file).height/rasterio.open(image_file).height)
    # print(xdem_min, ydem_min, xdem_max, ydem_max)

    dem_values = dem_data[math.floor(ydem_min):math.ceil(ydem_max), math.floor(xdem_min):math.ceil(xdem_max)]
    # print(dem_values)
    tree_height = np.max(dem_values) - np.min(dem_values)
  except:
    tree_height = np.nan
  return tree_height

def ReadDEM(dem_file):
  dem = rasterio.open(dem_file)
  dem_array = dem.read(1).astype('float64')
  return dem_array

def GetCrownSize(df, res):
  xmin = df.iloc[:,0]
  ymin = df.iloc[:,1]
  xmax = df.iloc[:,2]
  ymax = df.iloc[:,3]
  crown_size = 0.5*(res*(xmax - xmin) + abs(res)*(ymax - ymin))
  return crown_size

def ReturnTransformTIF(tif_file):
    '''
    This function takes in a path for tif file and returns dx, dy, x0, y0

    Arguments:
    1. tif_file: string: tif image file path

    Description:
    1. read the file line by line based on the description below, get dx, dy, x0, y0
    | x' |   | a  b  c | | x |
    | y' | = | d  e  f | | y |
    | 1  |   | 0  0  1 | | 1 |
    here a defines dx, c defines x0, e defines dy, f defines y0
    a, b, c, d, e, f
    0, 1, 2, 3, 4, 5
    Links:
    https://github.com/rasterio/affine
    https://rasterio.readthedocs.io/en/latest/topics/georeferencing.html

    Returns:
    1. dx (float)
    2. dy (float)
    3. x0 (float)
    4. y0(float)
    '''
    tifImage = rasterio.open(tif_file)#EPSG:3857, EPSG:32643
    dx = tifImage.transform[0]
    dy = tifImage.transform[4]
    x0 = tifImage.transform[2]
    y0 = tifImage.transform[5]
    return dx, dy, x0, y0

def ReturnTransformJGW(jgw_file):
    '''
    This function takes in a path for jgw file and returns dx, dy, x0, y0

    Arguments:
    1. jgw_file: string: jgw file path

    Description:
    1. read the file line by line based on the description below, get dx, dy, x0, y0
    Line 1: A: pixel size in the x-direction in map units/pixel
    Line 2: D: rotation about y-axis
    Line 3: B: rotation about x-axis
    Line 4: E: pixel size in the y-direction in map units, almost always negative[3]
    Line 5: C: x-coordinate of the center of the upper left pixel
    Line 6: F: y-coordinate of the center of the upper left pixel
    Link: https://en.wikipedia.org/wiki/World_file

    Returns:
    1. dx (float)
    2. dy (float)
    3. x0 (float)
    4. y0(float)
    '''
    with open(jgw_file) as f:
        lines = f.readlines()
    dx = float(lines[0].split('\n')[0])
    dy = float(lines[3].split('\n')[0])
    x0 = float(lines[4].split('\n')[0])
    y0 = float(lines[5].split('\n')[0])
    return dx, dy, x0, y0

def PlotPredictionRectangles(pred_lab_df, image_path):
  try:
    image = cv2.imread(image_path)
  except:
    image = image_path

  for index, row in pred_lab_df.dropna().iterrows():
    start_point_pred = (int(np.floor(row['xmin'])), int(np.floor(row['ymin'])))
    end_point_pred = (int(np.ceil(row['xmax'])), int(np.ceil(row['ymax'])))
    colorp = (0, 255, 0)
    thickness = 5
    image = cv2.rectangle(image, start_point_pred, end_point_pred, colorp, thickness)

  return image

def PlotSrNumbers(pred_lab_df, image_path):
  try:
    image = cv2.imread(image_path)
  except:
    image = image_path

  for index, row in pred_lab_df.dropna().iterrows():
    crown_center = (int(0.5*(row['xmin']+row['xmax'])), int(0.5*(row['ymin']+row['ymax'])))
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 1
    textcolor = (255, 255, 255)
    thickness = 5

    image = cv2.putText(image,
                        str(row['row_number']),
                        crown_center,
                        font,
                        fontScale,
                        textcolor,
                        thickness,
                        cv2.LINE_AA)

  return image

def GiveLowResVersion(image_path):
  im = Image.open(image_path)
  width, height = im.size
  target_width = 800
  scale_factor = target_width/width
  target_height = int(height*scale_factor)
  new_size = (target_width, target_height)
  im_resized = im.resize(new_size)
  return im_resized

def lat_lon_to_utm(latitude, longitude, zone):
    # Define the source and target coordinate systems
    source_proj = pyproj.Proj(proj='latlong', datum='WGS84')
    target_proj = pyproj.Proj(proj='utm', zone=zone, datum='WGS84')

    # Perform the conversion
    utm_easting, utm_northing = pyproj.transform(source_proj, target_proj, longitude, latitude)

    return utm_easting, utm_northing

def utm_to_lat_lon(utmx, utmy, zone):
    # Define the source and target coordinate systems
    target_proj = pyproj.Proj(proj='latlong', datum='WGS84')
    source_proj = pyproj.Proj(proj='utm', zone=zone, datum='WGS84')

    # Perform the conversion
    lat, longi = pyproj.transform(source_proj, target_proj, utmx, utmy)

    return utm_easting, utm_northing

# Ortho path, model path, converting 4 channel ortho to 3 and saving it somewhere to be used later

In [ ]:
path_to_saved_model = '/content/drive/MyDrive/Saved_Models/small_tree_15_iter.pl'
# orthomosaic_path = '/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Carbon Stock Estimation/Data/Drone Data/6_Testing Data/Ortho/5224211558394A494.tif'
# orthomosaic_path = "/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Carbon Stock Estimation/Data/Drone Data/1_Orthomosaic/Agroforestry/5224212558390A0408.jpg"
# orthomosaic_path = '/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Carbon Stock Estimation/Data/Drone Data/6_Testing Data/Ortho/275224211558390PVT08.tif'
# orthomosaic_path = '/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Carbon Stock Estimation/Deployment - Carbon Stock/ortho/275224211558363B292.tif'
# orthomosaic_path = '/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Carbon Stock Estimation/Deployment - Carbon Stock/ortho/275224211558394A495.tif'
# orthomosaic_path = '/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Carbon Stock Estimation/Deployment - Carbon Stock/ortho/275224212558410B1120.tif'
# orthomosaic_path = '/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Carbon Stock Estimation/Deployment - Carbon Stock/ortho/275224213558540B165.tif'
# orthomosaic_path = '/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Carbon Stock Estimation/Deployment - Carbon Stock/ortho/275224211558394A495.tif'
# orthomosaic_path = '/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Afforestation Dashboard/ortho/original/275224212558410B1120.tif'
orthomosaic_path = '/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Carbon Stock Estimation/Deployment - Carbon Stock/ortho/275214189555601B5134.jpg'
jgw_path = '/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Carbon Stock Estimation/Deployment - Carbon Stock/ortho/275214189555601B5134.jgw'

base_name = orthomosaic_path.split('/')[-1].split('.')[0]

image = plt.imread(orthomosaic_path)

if image.shape[-1] == 4:
  # @vijen: give some folder path in place of "'sample_data/'" to save a three channel image
  # generated after removing the alpha transparency channel
  image_path_for_tree_detection = 'sample_data/' + base_name + '.tif'
  image3ch = image[:,:,0:3]
  cv2.imwrite(image_path_for_tree_detection, image3ch[:,:,::-1])
else:
  image_path_for_tree_detection = orthomosaic_path

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Carbon Stock Estimation/Deployment - Carbon Stock/ortho/275214189555601B5134.jpg'

# Prediction and plotting predicted image

In [ ]:
test_type = "ttrained" #"ttrained" #"default"
# test_type = "default"

if test_type == "default":
  model = main.deepforest()
  model.use_release()
elif test_type == "ttrained":
  text_trap = io.StringIO()
  sys.stdout = text_trap
  model = main.deepforest.load_from_checkpoint(path_to_saved_model, verbose=False)
  sys.stdout = sys.__stdout__

score_thresh = 0.3
model.config["score_thresh"] = score_thresh
model.config["score"] = score_thresh
model.config['retinanet']['score_thresh'] = score_thresh

prediction_df, prediction_image = PredictTrees(model, image_path_for_tree_detection, 1500)

plt.imshow(prediction_image[:,:,:])

test = image[:,:,:]

# Get tree heights

In [ ]:
# @ Vijen: The DEM path is set to None in the beginning. If the user has uploaded a DEM, then set the Dem_path to the user uploaded DEM
dem_path = None
# dem_path = '/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Carbon Stock Estimation/Data/Drone Data/6_Testing Data/DSM/5224211558394A494.tif'
# dem_path = "/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Carbon Stock Estimation/Data/Drone Data/2_DEM/5224212558390A0408.tif"
# dem_path = "/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Carbon Stock Estimation/Data/Drone Data/2_DEM/275224211558390PVT08.tif"
# dem_path = '/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Carbon Stock Estimation/Deployment - Carbon Stock/dsm/275224211558363B292.tif'
# dem_path = '/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Carbon Stock Estimation/Deployment - Carbon Stock/dsm/275224211558394A495.tif'
# dem_path = '/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Carbon Stock Estimation/Deployment - Carbon Stock/dsm/275224212558410B1120.tif'
# dem_path = '/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Carbon Stock Estimation/Deployment - Carbon Stock/dsm/275224213558540B165.tif'
# dem_path = '/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Carbon Stock Estimation/Deployment - Carbon Stock/dsm/275224211558394A495.tif'
# dem_path = '/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Afforestation Dashboard/dem/275224212558410B1120.tif'
# dem_path = '/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Carbon Stock Estimation/Deployment - Carbon Stock/dsm/275214189555601B5134.tif'
dem_path = '/content/drive/MyDrive/Colab Notebooks/Data/Forest Tracking Tech/Current/Carbon Stock Estimation/Deployment - Carbon Stock/dsm/275214189555601B5134_new_for_demo.tif'

if dem_path == None:
  print("No dem file specified")
else:
  dem_data = ReadDEM(dem_path)
  prediction_df["Height"] = prediction_df[['xmin', 'ymin', 'xmax', 'ymax']].apply(GetTreeHeight, args=(dem_path, image_path_for_tree_detection, dem_data), axis=1)
  prediction_df["Height"][prediction_df["Height"]<0] = np.nan
  prediction_df["Height"][prediction_df["Height"]>50] = np.nan

# Get crown size

In [ ]:
# user needs to specify resolution in meter (that is amount of land length covered per pixel in meter)
# below, I am assuming the resolution to be 2.5 cm
res = 2.5/100
prediction_df["Crown"] = GetCrownSize(prediction_df[['xmin', 'ymin', 'xmax', 'ymax']], res)

# drop rows with nan values
prediction_df.dropna(inplace=True)

# Predict DBH

In [ ]:
dbh_model = "/content/drive/MyDrive/Colab Notebooks/Deployment/dbhm1.hdf"
scaler_model = '/content/drive/MyDrive/Colab Notebooks/Deployment/dbhs1.pkl'

loaded_model = load_model(dbh_model)
scaler = load(open(scaler_model, 'rb'))
prediction_df['TC_foot'] = prediction_df['Crown']*3.2
prediction_df['TH_foot'] = prediction_df['Height']*3.2
X_predict = prediction_df[['TH_foot', 'TC_foot']]
prediction_df['DBH_cm'] = loaded_model.predict(scaler.transform(X_predict))

# Calculate biomass

In [ ]:
prediction_df["Above ground Biomass"] = np.exp(-1.996 + 2.32*np.log(prediction_df["DBH_cm"]))
prediction_df["Total Biomass"] = 1.27*prediction_df["Above ground Biomass"]
prediction_df["Carbon Stock"] = 0.47*prediction_df["Total Biomass"]
prediction_df["CO2 Sequestration"] = (44/12)*prediction_df["Carbon Stock"]
total_CO2 = np.sum(prediction_df["CO2 Sequestration"])

In [ ]:
file_base_name = path.basename(orthomosaic_path)
file_name_without_ext, ext = path.splitext(path.basename(orthomosaic_path))

prediction_df["xcenter"] = (prediction_df["xmin"] + prediction_df["xmax"])/2
prediction_df["ycenter"] = (prediction_df["ymin"] + prediction_df["ymax"])/2

if (ext=='.tif'):
  dx, dy, x0, y0 = ReturnTransformTIF(orthomosaic_path)
  crs = str(rasterio.open(orthomosaic_path).crs)
  # myProj = Proj("+proj=utm +zone=43, +north +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
  myProj = Proj(proj='utm',zone=43,ellps='WGS84', preserve_units=False)
  if (str(rasterio.open(orthomosaic_path).crs) == 'EPSG:32643'):
    prediction_df["center_UTMx"] = x0 + dx * prediction_df["xcenter"]
    prediction_df["center_UTMy"] = y0 + dy * prediction_df["ycenter"]
    prediction_df["center_LONG"] = 0
    prediction_df["center_LAT"] = 0
    for index, row in prediction_df.iterrows():
      utmx = np.array(row["center_UTMx"]).reshape(1,)
      utmy = np.array(row["center_UTMy"]).reshape(1,)
      lon, lat = myProj(utmx, utmy, inverse = True)
      prediction_df.loc[index, ["center_LONG"]] = lon
      prediction_df.loc[index, ["center_LAT"]] = lat

  elif (str(rasterio.open(orthomosaic_path).crs) == 'EPSG:4326'):
    prediction_df["center_LONG"] = x0 + dx * prediction_df["xcenter"]
    prediction_df["center_LAT"] = y0 + dy * prediction_df["ycenter"]
    prediction_df["center_UTMx"] = 0
    prediction_df["center_UTMy"] = 0
    for index, row in prediction_df.iterrows():
      lat = np.array(row["center_LAT"]).reshape(1,)
      lon = np.array(row["center_LONG"]).reshape(1,)
      utm_easting, utm_northing = myProj(lon, lat)
      prediction_df.loc[index, ["center_UTMx"]] = utm_easting
      prediction_df.loc[index, ["center_UTMy"]] = utm_northing
elif (ext=='.jpg'):
  dx, dy, x0, y0 = ReturnTransformJGW(jgw_path)
  myProj = Proj(proj='utm',zone=43,ellps='WGS84', preserve_units=False)
  if (y0>90):
    prediction_df["center_UTMx"] = x0 + dx * prediction_df["xcenter"]
    prediction_df["center_UTMy"] = y0 + dy * prediction_df["ycenter"]
    prediction_df["center_LONG"] = 0
    prediction_df["center_LAT"] = 0
    for index, row in prediction_df.iterrows():
      utmx = np.array(row["center_UTMx"]).reshape(1,)
      utmy = np.array(row["center_UTMy"]).reshape(1,)
      lon, lat = myProj(utmx, utmy, inverse = True)
      prediction_df.loc[index, ["center_LONG"]] = lon
      prediction_df.loc[index, ["center_LAT"]] = lat

  elif (y0<90):
    prediction_df["center_LONG"] = x0 + dx * prediction_df["xcenter"]
    prediction_df["center_LAT"] = y0 + dy * prediction_df["ycenter"]
    prediction_df["center_UTMx"] = 0
    prediction_df["center_UTMy"] = 0
    for index, row in prediction_df.iterrows():
      lat = np.array(row["center_LAT"]).reshape(1,)
      lon = np.array(row["center_LONG"]).reshape(1,)
      utm_easting, utm_northing = myProj(lon, lat)
      prediction_df.loc[index, ["center_UTMx"]] = utm_easting
      prediction_df.loc[index, ["center_UTMy"]] = utm_northing


prediction_df["xcenter"] = prediction_df["xcenter"].astype(int)
prediction_df["ycenter"] = prediction_df["ycenter"].astype(int)
prediction_df = prediction_df.sort_values(["xcenter", "ycenter"])
prediction_df = prediction_df.assign(row_number=range(len(prediction_df)))

prediction_with_rectrangles = PlotPredictionRectangles(prediction_df, orthomosaic_path)
plt.imshow(prediction_with_rectrangles[:,:,::-1])
plt.show()
try:
  prediction_with_rectrangles_RGBA =  np.concatenate([prediction_with_rectrangles[:,:,::-1], image[:,:,3].reshape(image.shape[0], image.shape[1],1)], axis=2)
except:
  prediction_with_rectrangles_RGBA = prediction_with_rectrangles[:,:,::-1].copy()

plt.imshow(prediction_with_rectrangles_RGBA)
plt.show()
plt.imsave('/content/drive/MyDrive/Colab Notebooks/Deployment/' + base_name + '_rect.png', prediction_with_rectrangles_RGBA)

prediction_with_rectrangles_numbers_RGBA = PlotSrNumbers(prediction_df, '/content/drive/MyDrive/Colab Notebooks/Deployment/' + base_name + '_rect.png')
plt.imshow(prediction_with_rectrangles_numbers_RGBA)
plt.show()
plt.imsave('/content/drive/MyDrive/Colab Notebooks/Deployment/' + base_name + '_rect_num.png', prediction_with_rectrangles_numbers_RGBA)

plt.imshow(dem_data)
plt.show()

prediction_with_rectrangles_numbers_RGBA_low_res = GiveLowResVersion('/content/drive/MyDrive/Colab Notebooks/Deployment/' + base_name + '_rect_num.png')
prediction_with_rectrangles_numbers_RGBA_low_res.save('/content/drive/MyDrive/Colab Notebooks/Deployment/' + base_name + 'rect_num_low_res.png')
prediction_with_rectrangles_RGBA_low_res = GiveLowResVersion('/content/drive/MyDrive/Colab Notebooks/Deployment/' + base_name + '_rect.png')
prediction_with_rectrangles_RGBA_low_res.save('/content/drive/MyDrive/Colab Notebooks/Deployment/' + base_name + 'rect_low_res.png')
prediction_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Deployment/' + base_name + '.csv')
# # print(rasterio.open(orthomosaic_path).transform)
# # print(type(rasterio.open(orthomosaic_path).crs))
# # print(rasterio.open(orthomosaic_path).meta)
# a = str(rasterio.open(orthomosaic_path).crs)

In [ ]:
!pip list  > '/content/drive/MyDrive/Colab Notebooks/Deployment/package_list.txt'

In [ ]:
!pip freeze > '/content/drive/MyDrive/Colab Notebooks/Deployment/package_list.txt'

In [ ]:
# !pip install -r '/content/drive/MyDrive/Colab Notebooks/Deployment/package_list.txt'

In [ ]:
# import types
# def imports():
#     for name, val in globals().items():
#         if isinstance(val, types.ModuleType):
#             yield val.__name__
# list(imports())

In [ ]:
plt.imshow(prediction_with_rectrangles_RGBA)